In [100]:
# import libraries

import pandas as pd
import numpy as np
from statistics import geometric_mean

In [101]:
# import collection set

collection = pd.read_csv("The Collection Set.csv")
collection = collection[["NAME", "Status", "Variables"]]
collection['NAME'] = collection['NAME'].astype(str) + '.mps.gz'

In [102]:
# import result sets

data_no_cuts = pd.read_csv("results_0.csv")
data_cuts = pd.read_csv("results_1.csv")

In [103]:
# SET 1
# All 10 runs solve before reaching time limit

set_1_no_cuts = data_no_cuts[(data_no_cuts["SOLUTION TIME"] < 18000)]
max_repeat = 5
vc = set_1_no_cuts['NAME'].value_counts()
set_1_no_cuts = set_1_no_cuts[set_1_no_cuts['NAME'].isin(vc[vc >= max_repeat].index)]


set_1_cuts = data_cuts[(data_cuts["SOLUTION TIME"] < 18000)]
max_repeat = 5
c = set_1_cuts['NAME'].value_counts()
set_1_cuts = set_1_cuts[set_1_cuts['NAME'].isin(c[c >= max_repeat].index)]

In [104]:
set_1_no_cuts = set_1_no_cuts.groupby('NAME').mean()
set_1_cuts = set_1_cuts.groupby('NAME').mean()

set_1_no_cuts = set_1_no_cuts[["SOLUTION TIME", "No. OF NODES"]]
set_1_cuts = set_1_cuts[["SOLUTION TIME", "No. OF NODES"]]

set_1_no_cuts.rename(columns={'SOLUTION TIME': 'TIME W/O CUTS', 'No. OF NODES': 'NODES W/O CUTS'}, inplace=True)
set_1_cuts.rename(columns={'SOLUTION TIME': 'TIME W/ CUTS', 'No. OF NODES': 'NODES W/ CUTS'}, inplace=True)

In [105]:
analysis_set_1 = set_1_no_cuts.merge(set_1_cuts, on = "NAME", how = "left")
analysis_set_1 = analysis_set_1.merge(collection, on = "NAME", how = "left")
analysis_set_1 = analysis_set_1.dropna()

analysis_set_1['Minimum Time'] = analysis_set_1[['TIME W/O CUTS', 'TIME W/ CUTS']].min(axis = 1)
analysis_set_1


,NAME,TIME W/O CUTS,NODES W/O CUTS,TIME W/ CUTS,NODES W/ CUTS,Status,Variables,Minimum Time
0,10teams.mps.gz,12.386720,276.0,10.997772,30.0,easy,2025.0,10.997772
1,22433.mps.gz,0.650582,5.0,2.535930,1.0,easy,429.0,0.650582
2,23588.mps.gz,2.399748,981.0,5.527370,576.0,easy,368.0,2.399748
3,30_70_45_05_100.mps.gz,240.079782,201.0,282.653795,46.0,easy,10772.0,240.079782
4,30_70_45_095_100.mps.gz,55.714462,1.0,743.997969,201.0,easy,10976.0,55.714462
...,...,...,...,...,...,...,...,...
365,triptim1.mps.gz,690.691198,11.0,2446.382658,78.0,easy,30055.0,690.691198
366,umts.mps.gz,11604.852368,3106370.0,1079.840992,144227.0,easy,2947.0,1079.840992
367,unitcal_7.mps.gz,10563.733016,196989.0,940.799926,705.0,easy,25755.0,940.799926
368,var-smallemery-m6j6.mps.gz,10246.224785,836578.0,7171.038360,685889.0,easy,5608.0,7171.038360


In [106]:
# bucket_min = [0, 10, 100, 1000, 3600]
# MAX_TIME = 3600*5
# bucket_max = [MAX_TIME]*len(bucket_min)

# conditions = []
# values = []
# for min_time, max_time in zip(bucket_min,bucket_max):
#     conditions.append(
#         (min_time <= analysis_set_1['TIME W/O CUTS']) 
#         & (analysis_set_1['TIME W/O CUTS'] <= max_time) 
#         & (min_time <= analysis_set_1['TIME W/ CUTS']) 
#         & (analysis_set_1['TIME W/ CUTS'] <= max_time)
#     )
#     values.append("[{}, {})".format(min_time, max_time))

# conditions.reverse()
# values.reverse()
# analysis_set_1['BUCKET'] = np.select(conditions, values)

# conditions.reverse()
# values.reverse()

# analysis_set_1
# # analysis_set_1.to_csv('analysis_set_1.csv')

In [107]:
SHIFT_TIME  = 60
SHIFT_NODES = 1000

df_0 = analysis_set_1[(analysis_set_1['Minimum Time'] >= 0)]
df_10 = analysis_set_1[(analysis_set_1['Minimum Time'] >= 10)]
df_100 = analysis_set_1[(analysis_set_1['Minimum Time'] >= 100)]
df_1000 = analysis_set_1[(analysis_set_1['Minimum Time'] >= 1000)]
df_3600 = analysis_set_1[(analysis_set_1['Minimum Time'] >= 3600)]

In [108]:
# 0 - 18000

time_no_cuts_0 = df_0['TIME W/O CUTS'].tolist()
nodes_no_cuts_0 = df_0['NODES W/O CUTS'].tolist()
    
time_cuts_0 = df_0['TIME W/ CUTS'].tolist()
nodes_cuts_0 = df_0['NODES W/ CUTS'].tolist()
    
timeshift_no_cuts_0 = [i + SHIFT_TIME for i in time_no_cuts_0]
nodeshift_no_cuts_0 = [j + SHIFT_NODES for j in nodes_no_cuts_0]
    
timeshift_cuts_0 = [i + SHIFT_TIME for i in time_cuts_0]
nodeshift_cuts_0 = [j + SHIFT_NODES for j in nodes_cuts_0]

instances_0 = len(df_0)

time_no_cuts_0 = []
nodes_no_cuts_0 = []
cuts_geom_time_0 = geometric_mean(timeshift_no_cuts_0) - SHIFT_TIME
cuts_geom_node_0 = geometric_mean(nodeshift_no_cuts_0) - SHIFT_NODES
time_no_cuts_0.append(cuts_geom_time_0)
nodes_no_cuts_0.append(cuts_geom_node_0)

time_cuts_0 = []
nodes_cuts_0 = []
cuts_geom_time_0 = geometric_mean(timeshift_cuts_0) - SHIFT_TIME
cuts_geom_node_0 = geometric_mean(nodeshift_cuts_0) - SHIFT_NODES
time_cuts_0.append(cuts_geom_time_0)
nodes_cuts_0.append(cuts_geom_node_0)

# print(instances_0)
# print(time_no_cuts_0)
# print(nodes_no_cuts_0)
# print(time_cuts_0)
# print(nodes_cuts_0)

# 10 - 18000
time_no_cuts_10 = df_10['TIME W/O CUTS'].tolist()
nodes_no_cuts_10 = df_10['NODES W/O CUTS'].tolist()
    
time_cuts_10 = df_10['TIME W/ CUTS'].tolist()
nodes_cuts_10 = df_10['NODES W/ CUTS'].tolist()
    
timeshift_no_cuts_10 = [i + SHIFT_TIME for i in time_no_cuts_10]
nodeshift_no_cuts_10 = [j + SHIFT_NODES for j in nodes_no_cuts_10]
    
timeshift_cuts_10 = [i + SHIFT_TIME for i in time_cuts_10]
nodeshift_cuts_10 = [j + SHIFT_NODES for j in nodes_cuts_10]

instances_10 = len(df_10)

time_no_cuts_10 = []
nodes_no_cuts_10 = []
cuts_geom_time_10 = geometric_mean(timeshift_no_cuts_10) - SHIFT_TIME
cuts_geom_node_10 = geometric_mean(nodeshift_no_cuts_10) - SHIFT_NODES
time_no_cuts_10.append(cuts_geom_time_10)
nodes_no_cuts_10.append(cuts_geom_node_10)

time_cuts_10 = []
nodes_cuts_10 = []
cuts_geom_time_10 = geometric_mean(timeshift_cuts_10) - SHIFT_TIME
cuts_geom_node_10 = geometric_mean(nodeshift_cuts_10) - SHIFT_NODES
time_cuts_10.append(cuts_geom_time_10)
nodes_cuts_10.append(cuts_geom_node_10)

# print(instances_10)
# print(time_no_cuts_10)
# print(nodes_no_cuts_10)
# print(time_cuts_10)
# print(nodes_cuts_10)

# 100 - 18000
time_no_cuts_100 = df_100['TIME W/O CUTS'].tolist()
nodes_no_cuts_100 = df_100['NODES W/O CUTS'].tolist()
    
time_cuts_100 = df_100['TIME W/ CUTS'].tolist()
nodes_cuts_100 = df_100['NODES W/ CUTS'].tolist()
    
timeshift_no_cuts_100 = [i + SHIFT_TIME for i in time_no_cuts_100]
nodeshift_no_cuts_100 = [j + SHIFT_NODES for j in nodes_no_cuts_100]
    
timeshift_cuts_100 = [i + SHIFT_TIME for i in time_cuts_100]
nodeshift_cuts_100 = [j + SHIFT_NODES for j in nodes_cuts_100]

instances_100 = len(df_100)

time_no_cuts_100 = []
nodes_no_cuts_100 = []
cuts_geom_time_100 = geometric_mean(timeshift_no_cuts_100) - SHIFT_TIME
cuts_geom_node_100 = geometric_mean(nodeshift_no_cuts_100) - SHIFT_NODES
time_no_cuts_100.append(cuts_geom_time_100)
nodes_no_cuts_100.append(cuts_geom_node_100)

time_cuts_100 = []
nodes_cuts_100 = []
cuts_geom_time_100 = geometric_mean(timeshift_cuts_100) - SHIFT_TIME
cuts_geom_node_100 = geometric_mean(nodeshift_cuts_100) - SHIFT_NODES
time_cuts_100.append(cuts_geom_time_100)
nodes_cuts_100.append(cuts_geom_node_100)

# print(instances_100)
# print(time_no_cuts_100)
# print(nodes_no_cuts_100)
# print(time_cuts_100)
# print(nodes_cuts_100)

# 1000 - 18000
time_no_cuts_1000 = df_1000['TIME W/O CUTS'].tolist()
nodes_no_cuts_1000 = df_1000['NODES W/O CUTS'].tolist()
    
time_cuts_1000 = df_1000['TIME W/ CUTS'].tolist()
nodes_cuts_1000 = df_1000['NODES W/ CUTS'].tolist()
    
timeshift_no_cuts_1000 = [i + SHIFT_TIME for i in time_no_cuts_1000]
nodeshift_no_cuts_1000 = [j + SHIFT_NODES for j in nodes_no_cuts_1000]
    
timeshift_cuts_1000 = [i + SHIFT_TIME for i in time_cuts_1000]
nodeshift_cuts_1000 = [j + SHIFT_NODES for j in nodes_cuts_1000]

instances_1000 = len(df_1000)

time_no_cuts_1000 = []
nodes_no_cuts_1000 = []
cuts_geom_time_1000 = geometric_mean(timeshift_no_cuts_1000) - SHIFT_TIME
cuts_geom_node_1000 = geometric_mean(nodeshift_no_cuts_1000) - SHIFT_NODES
time_no_cuts_1000.append(cuts_geom_time_1000)
nodes_no_cuts_1000.append(cuts_geom_node_1000)

time_cuts_1000 = []
nodes_cuts_1000 = []
cuts_geom_time_1000 = geometric_mean(timeshift_cuts_1000) - SHIFT_TIME
cuts_geom_node_1000 = geometric_mean(nodeshift_cuts_1000) - SHIFT_NODES
time_cuts_1000.append(cuts_geom_time_1000)
nodes_cuts_1000.append(cuts_geom_node_1000)

# print(instances_1000)
# print(time_no_cuts_1000)
# print(nodes_no_cuts_1000)
# print(time_cuts_1000)
# print(nodes_cuts_1000)

# 3600 - 18000
time_no_cuts_3600 = df_3600['TIME W/O CUTS'].tolist()
nodes_no_cuts_3600 = df_3600['NODES W/O CUTS'].tolist()
    
time_cuts_3600 = df_3600['TIME W/ CUTS'].tolist()
nodes_cuts_3600 = df_3600['NODES W/ CUTS'].tolist()
    
timeshift_no_cuts_3600 = [i + SHIFT_TIME for i in time_no_cuts_3600]
nodeshift_no_cuts_3600 = [j + SHIFT_NODES for j in nodes_no_cuts_3600]
    
timeshift_cuts_3600 = [i + SHIFT_TIME for i in time_cuts_3600]
nodeshift_cuts_3600 = [j + SHIFT_NODES for j in nodes_cuts_3600]

instances_3600 = len(df_3600)

time_no_cuts_3600 = []
nodes_no_cuts_3600 = []
cuts_geom_time_3600 = geometric_mean(timeshift_no_cuts_3600) - SHIFT_TIME
cuts_geom_node_3600 = geometric_mean(nodeshift_no_cuts_3600) - SHIFT_NODES
time_no_cuts_3600.append(cuts_geom_time_3600)
nodes_no_cuts_3600.append(cuts_geom_node_3600)

time_cuts_3600 = []
nodes_cuts_3600 = []
cuts_geom_time_3600 = geometric_mean(timeshift_cuts_3600) - SHIFT_TIME
cuts_geom_node_3600 = geometric_mean(nodeshift_cuts_3600) - SHIFT_NODES
time_cuts_3600.append(cuts_geom_time_3600)
nodes_cuts_3600.append(cuts_geom_node_3600)

instances = []
instances.append(instances_0)
instances.append(instances_10)
instances.append(instances_100)
instances.append(instances_1000)
instances.append(instances_3600)

time_no_cuts = time_no_cuts_0 + time_no_cuts_10 + time_no_cuts_100 + time_no_cuts_1000 + time_no_cuts_3600
node_no_cuts = nodes_no_cuts_0 + nodes_no_cuts_10 + nodes_no_cuts_100 + nodes_no_cuts_1000 + nodes_no_cuts_3600

time_cuts = time_cuts_0 + time_cuts_10 + time_cuts_100 + time_cuts_1000 + time_cuts_3600  
node_cuts = nodes_cuts_0 + nodes_cuts_10 + nodes_cuts_100 + nodes_cuts_1000 + nodes_cuts_3600

print(instances)
print(time_cuts)
print(time_no_cuts)

[355, 259, 159, 54, 14]
[196.686439924725, 368.2972667768399, 910.567439740418, 3474.787122869382, 7946.551772431248]
[244.8238098251125, 443.9158752273817, 1031.4959003308186, 3820.2968747487903, 9688.014674108455]


In [109]:
bb_classes = ['Geometric Mean']
num_bb_classes = len(bb_classes)

bucket_min = [0, 10, 100, 1000, 3600]
bucket_max = [18000, 18000, 18000, 18000, 18000]
num_buckets = len(bucket_min)
assert(len(bucket_max) == num_buckets)
bb_buckets = ['[' + str(bucket_min[j]) + ',' + str(bucket_max[j]) + ')' for j in range(num_buckets)]

bb_metrics = ['Gmean']

In [110]:
avg_bb_cols = pd.MultiIndex.from_product([['Time','Node'],
                                     ['Cut','No Cut']],
                                    names=['Parameter','Setting'])

bb_row_names = pd.MultiIndex.from_product(
    [bb_classes, bb_buckets],
    names=['Metric', 'Bucket'])

avg_bb_df = pd.DataFrame(
    index = bb_row_names,
    dtype = float)

avg_bb_df = pd.DataFrame(
    columns = avg_bb_cols,
    index = bb_row_names,
    dtype = float)

avg_bb_df['Instances'] = instances

avg_bb_df[('Time', 'Cut')] = time_cuts
avg_bb_df[('Time', 'No Cut')] = time_no_cuts
avg_bb_df[('Node', 'Cut')] = node_cuts
avg_bb_df[('Node', 'No Cut')] = node_no_cuts


avg_bb_df

Parameter                           Time                       Node  \
Setting                              Cut       No Cut           Cut   
Metric         Bucket                                                 
Geometric Mean [0,18000)      196.686440   244.823810   3106.453952   
               [10,18000)     368.297267   443.915875   5605.053776   
               [100,18000)    910.567440  1031.495900  10475.199677   
               [1000,18000)  3474.787123  3820.296875  47286.860657   
               [3600,18000)  7946.551772  9688.014674  92018.347853   

Parameter                                  Instances  
Setting                             No Cut            
Metric         Bucket                                 
Geometric Mean [0,18000)       7967.012311       355  
               [10,18000)     13840.151057       259  
               [100,18000)    21777.521319       159  
               [1000,18000)  100219.336515        54  
               [3600,18000)  192284.003111        14

In [111]:
# SET 2
# out of 10 runs at least 1 and at most 9 runs timeout

set_2_no_cuts = data_no_cuts
max_repeat = 5
vc = set_2_no_cuts['NAME'].value_counts()
set_2_no_cuts = set_2_no_cuts[set_2_no_cuts['NAME'].isin(vc[vc >= max_repeat].index)]


set_2_cuts = data_cuts
max_repeat = 5
c = set_2_cuts['NAME'].value_counts()
set_2_cuts = set_2_cuts[set_2_cuts['NAME'].isin(c[c >= max_repeat].index)]

set_2 = pd.concat([set_2_cuts, set_2_no_cuts])
set_2 = set_2.sort_values(by = ['NAME', 'CUT SETTING'], ascending = True)

counts = set_2['NAME'].value_counts()
res = set_2[~set_2['NAME'].isin(counts[counts < 10].index)]

dic = dict()
for index, row in res.iterrows():
    if row['SOLUTION TIME'] > 18000:
        if row['NAME'] in dic:
            dic[row['NAME']] += 1
        else:
            dic[row["NAME"]] = 1
        if dic[row['NAME']] == 10:
            del dic[row['NAME']]
# dic
set_2_no_cuts = set_2_no_cuts[set_2_no_cuts['NAME'].isin(dic.keys())]
set_2_cuts = set_2_cuts[set_2_cuts['NAME'].isin(dic.keys())]

set_2_cuts

,NAME,INITIAL LP,FIRST ROUND CUT,ROUND OF CUTS,OBJECTIVE VALUE ROOT NODE,BEST SOLUTION,SOLUTION TIME,PRESOLVING TIME,BEST PRIMAL BOUND,GAP,No. OF CUTS GENERATED,No. OF CUTS APPLIED,No. OF CUTS PRESENT IN LP AT THE END,No. OF NODES,RANDOM SEED,CUT SETTING,STATUS,END
170,arki001.mps.gz,1273480.9,7579621.852,28,7.580142e+06,7580813.065,18000.313650,0.108462,7580813.065,0.00001,134979380,792219,0,2268704,2,1,timelimit,DONE
171,arki001.mps.gz,1273480.9,7579621.852,28,7.580142e+06,7580813.065,18002.197530,0.118306,7580813.065,0.00001,138535054,792219,0,2268815,4,1,timelimit,DONE
172,arki001.mps.gz,1273480.9,7579621.852,28,7.580142e+06,7580813.065,18000.109870,0.116983,7580813.065,0.00001,139794870,792219,0,2268854,8,1,timelimit,DONE
173,arki001.mps.gz,1273480.9,7579621.852,28,7.580142e+06,7580813.065,18000.784610,0.111195,7580813.065,0.00001,149319287,792219,0,2269143,16,1,timelimit,DONE
174,arki001.mps.gz,1273480.9,7579621.852,28,7.580142e+06,7580813.065,18001.405890,0.113116,7580813.065,0.00001,148013678,792219,0,2269104,32,1,timelimit,DONE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3519,uct-subprob.mps.gz,-2.0,244.000,52,2.757333e+02,314.000,3242.629652,0.154356,314.000,0.00000,1089107,47757,0,75300,2,1,optimal,DONE
3520,uct-subprob.mps.gz,-2.0,244.000,52,2.757333e+02,314.000,3217.745203,0.145398,314.000,0.00000,1089107,47757,0,75300,4,1,optimal,DONE
3521,uct-subprob.mps.gz,-2.0,244.000,52,2.757333e+02,314.000,3207.929860,0.149388,314.000,0.00000,1089107,47757,0,75300,8,1,optimal,DONE
3522,uct-subprob.mps.gz,-2.0,244.000,52,2.757333e+02,314.000,3157.883368,0.157537,314.000,0.00000,1089107,47757,0,75300,16,1,optimal,DONE


In [112]:
time_cuts = []
for index, row in set_2_cuts.iterrows():
    if row['SOLUTION TIME'] >= 18000:
        time_cuts.append(row['SOLUTION TIME'] * 5)
    else:
        time_cuts.append(row['SOLUTION TIME'])

time_no_cuts = []
for index, row in set_2_no_cuts.iterrows():
    if row['SOLUTION TIME'] >= 18000:
        time_no_cuts.append(row['SOLUTION TIME'] * 5)
    else:
        time_no_cuts.append(row['SOLUTION TIME'])
        
set_2_cuts['TIME W/ CUTS'] = time_cuts
set_2_no_cuts['TIME W/O CUTS'] = time_no_cuts

In [113]:
set_2_no_cuts = set_2_no_cuts.groupby('NAME').mean()
set_2_cuts = set_2_cuts.groupby('NAME').mean()

set_2_no_cuts = set_2_no_cuts[["TIME W/O CUTS", "No. OF NODES"]]
set_2_cuts = set_2_cuts[["TIME W/ CUTS", "No. OF NODES"]]

set_2_no_cuts.rename(columns={'No. OF NODES': 'NODES W/O CUTS'}, inplace=True)
set_2_cuts.rename(columns={'No. OF NODES': 'NODES W/ CUTS'}, inplace=True)

set_2_cuts

,TIME W/ CUTS,NODES W/ CUTS
NAME,,
CMS750_4.mps.gz,90002.461770,1.0
arki001.mps.gz,90004.811550,2268924.0
atlanta-ip.mps.gz,15300.894714,55649.0
bab5.mps.gz,13413.205184,204088.0
brazil3.mps.gz,15227.835190,4669.0
...,...,...
sorrell3.mps.gz,45936.878390,33190.0
sp98ar.mps.gz,15516.920478,192153.0
toll-like.mps.gz,16718.294914,360893.0


In [114]:
analysis_set_2 = set_2_no_cuts.merge(set_2_cuts, on = "NAME", how = "left")
analysis_set_2 = analysis_set_2.merge(collection, on = "NAME", how = "left")
analysis_set_2 = analysis_set_2.dropna()
analysis_set_2

,NAME,TIME W/O CUTS,NODES W/O CUTS,TIME W/ CUTS,NODES W/ CUTS,Status,Variables
0,CMS750_4.mps.gz,15883.387148,155687.0,90002.461770,1.0,easy,11697.0
1,arki001.mps.gz,3197.485577,1761208.0,90004.811550,2268924.0,easy,1388.0
2,atlanta-ip.mps.gz,75529.082922,126363.8,15300.894714,55649.0,easy,48738.0
3,bab5.mps.gz,90000.025370,381053.0,13413.205184,204088.0,easy,21600.0
4,brazil3.mps.gz,90000.009490,11584.8,15227.835190,4669.0,easy,23968.0
...,...,...,...,...,...,...,...
66,sorrell3.mps.gz,90000.032540,813520.8,45936.878390,33190.0,hard,1024.0
67,sp98ar.mps.gz,90000.025040,593980.4,15516.920478,192153.0,easy,15085.0
68,toll-like.mps.gz,90000.218220,3529829.2,16718.294914,360893.0,easy,2883.0
69,traininstance6.mps.gz,12958.168982,3628796.0,90000.082150,6008000.2,easy,10218.0


In [115]:
analysis_set_2['Minimum Time'] = analysis_set_2[['TIME W/O CUTS', 'TIME W/ CUTS']].min(axis = 1)
analysis_set_2
# analysis_set_2.to_csv('analysis_set_2.csv')

In [116]:
SHIFT_TIME  = 60
SHIFT_NODES = 1000

df_0 = analysis_set_2[(analysis_set_2['Minimum Time'] >= 0)]
df_10 = analysis_set_2[(analysis_set_2['Minimum Time'] >= 10)]
df_100 = analysis_set_2[(analysis_set_2['Minimum Time'] >= 100)]
df_1000 = analysis_set_2[(analysis_set_2['Minimum Time'] >= 1000)]
df_3600 = analysis_set_2[(analysis_set_2['Minimum Time'] >= 3600)]

In [117]:
# 0 - 18000

time_no_cuts_0 = df_0['TIME W/O CUTS'].tolist()
nodes_no_cuts_0 = df_0['NODES W/O CUTS'].tolist()
    
time_cuts_0 = df_0['TIME W/ CUTS'].tolist()
nodes_cuts_0 = df_0['NODES W/ CUTS'].tolist()
    
timeshift_no_cuts_0 = [i + SHIFT_TIME for i in time_no_cuts_0]
nodeshift_no_cuts_0 = [j + SHIFT_NODES for j in nodes_no_cuts_0]
    
timeshift_cuts_0 = [i + SHIFT_TIME for i in time_cuts_0]
nodeshift_cuts_0 = [j + SHIFT_NODES for j in nodes_cuts_0]

instances_0 = len(df_0)

time_no_cuts_0 = []
nodes_no_cuts_0 = []
cuts_geom_time_0 = geometric_mean(timeshift_no_cuts_0) - SHIFT_TIME
cuts_geom_node_0 = geometric_mean(nodeshift_no_cuts_0) - SHIFT_NODES
time_no_cuts_0.append(cuts_geom_time_0)
nodes_no_cuts_0.append(cuts_geom_node_0)

time_cuts_0 = []
nodes_cuts_0 = []
cuts_geom_time_0 = geometric_mean(timeshift_cuts_0) - SHIFT_TIME
cuts_geom_node_0 = geometric_mean(nodeshift_cuts_0) - SHIFT_NODES
time_cuts_0.append(cuts_geom_time_0)
nodes_cuts_0.append(cuts_geom_node_0)

# print(instances_0)
# print(time_no_cuts_0)
# print(nodes_no_cuts_0)
# print(time_cuts_0)
# print(nodes_cuts_0)

# 10 - 18000
time_no_cuts_10 = df_10['TIME W/O CUTS'].tolist()
nodes_no_cuts_10 = df_10['NODES W/O CUTS'].tolist()
    
time_cuts_10 = df_10['TIME W/ CUTS'].tolist()
nodes_cuts_10 = df_10['NODES W/ CUTS'].tolist()
    
timeshift_no_cuts_10 = [i + SHIFT_TIME for i in time_no_cuts_10]
nodeshift_no_cuts_10 = [j + SHIFT_NODES for j in nodes_no_cuts_10]
    
timeshift_cuts_10 = [i + SHIFT_TIME for i in time_cuts_10]
nodeshift_cuts_10 = [j + SHIFT_NODES for j in nodes_cuts_10]

instances_10 = len(df_10)

time_no_cuts_10 = []
nodes_no_cuts_10 = []
cuts_geom_time_10 = geometric_mean(timeshift_no_cuts_10) - SHIFT_TIME
cuts_geom_node_10 = geometric_mean(nodeshift_no_cuts_10) - SHIFT_NODES
time_no_cuts_10.append(cuts_geom_time_10)
nodes_no_cuts_10.append(cuts_geom_node_10)

time_cuts_10 = []
nodes_cuts_10 = []
cuts_geom_time_10 = geometric_mean(timeshift_cuts_10) - SHIFT_TIME
cuts_geom_node_10 = geometric_mean(nodeshift_cuts_10) - SHIFT_NODES
time_cuts_10.append(cuts_geom_time_10)
nodes_cuts_10.append(cuts_geom_node_10)

# print(instances_10)
# print(time_no_cuts_10)
# print(nodes_no_cuts_10)
# print(time_cuts_10)
# print(nodes_cuts_10)

# 100 - 18000
time_no_cuts_100 = df_100['TIME W/O CUTS'].tolist()
nodes_no_cuts_100 = df_100['NODES W/O CUTS'].tolist()
    
time_cuts_100 = df_100['TIME W/ CUTS'].tolist()
nodes_cuts_100 = df_100['NODES W/ CUTS'].tolist()
    
timeshift_no_cuts_100 = [i + SHIFT_TIME for i in time_no_cuts_100]
nodeshift_no_cuts_100 = [j + SHIFT_NODES for j in nodes_no_cuts_100]
    
timeshift_cuts_100 = [i + SHIFT_TIME for i in time_cuts_100]
nodeshift_cuts_100 = [j + SHIFT_NODES for j in nodes_cuts_100]

instances_100 = len(df_100)

time_no_cuts_100 = []
nodes_no_cuts_100 = []
cuts_geom_time_100 = geometric_mean(timeshift_no_cuts_100) - SHIFT_TIME
cuts_geom_node_100 = geometric_mean(nodeshift_no_cuts_100) - SHIFT_NODES
time_no_cuts_100.append(cuts_geom_time_100)
nodes_no_cuts_100.append(cuts_geom_node_100)

time_cuts_100 = []
nodes_cuts_100 = []
cuts_geom_time_100 = geometric_mean(timeshift_cuts_100) - SHIFT_TIME
cuts_geom_node_100 = geometric_mean(nodeshift_cuts_100) - SHIFT_NODES
time_cuts_100.append(cuts_geom_time_100)
nodes_cuts_100.append(cuts_geom_node_100)

# print(instances_100)
# print(time_no_cuts_100)
# print(nodes_no_cuts_100)
# print(time_cuts_100)
# print(nodes_cuts_100)

# 1000 - 18000
time_no_cuts_1000 = df_1000['TIME W/O CUTS'].tolist()
nodes_no_cuts_1000 = df_1000['NODES W/O CUTS'].tolist()
    
time_cuts_1000 = df_1000['TIME W/ CUTS'].tolist()
nodes_cuts_1000 = df_1000['NODES W/ CUTS'].tolist()
    
timeshift_no_cuts_1000 = [i + SHIFT_TIME for i in time_no_cuts_1000]
nodeshift_no_cuts_1000 = [j + SHIFT_NODES for j in nodes_no_cuts_1000]
    
timeshift_cuts_1000 = [i + SHIFT_TIME for i in time_cuts_1000]
nodeshift_cuts_1000 = [j + SHIFT_NODES for j in nodes_cuts_1000]

instances_1000 = len(df_1000)

time_no_cuts_1000 = []
nodes_no_cuts_1000 = []
cuts_geom_time_1000 = geometric_mean(timeshift_no_cuts_1000) - SHIFT_TIME
cuts_geom_node_1000 = geometric_mean(nodeshift_no_cuts_1000) - SHIFT_NODES
time_no_cuts_1000.append(cuts_geom_time_1000)
nodes_no_cuts_1000.append(cuts_geom_node_1000)

time_cuts_1000 = []
nodes_cuts_1000 = []
cuts_geom_time_1000 = geometric_mean(timeshift_cuts_1000) - SHIFT_TIME
cuts_geom_node_1000 = geometric_mean(nodeshift_cuts_1000) - SHIFT_NODES
time_cuts_1000.append(cuts_geom_time_1000)
nodes_cuts_1000.append(cuts_geom_node_1000)

# print(instances_1000)
# print(time_no_cuts_1000)
# print(nodes_no_cuts_1000)
# print(time_cuts_1000)
# print(nodes_cuts_1000)

# 3600 - 18000
time_no_cuts_3600 = df_3600['TIME W/O CUTS'].tolist()
nodes_no_cuts_3600 = df_3600['NODES W/O CUTS'].tolist()
    
time_cuts_3600 = df_3600['TIME W/ CUTS'].tolist()
nodes_cuts_3600 = df_3600['NODES W/ CUTS'].tolist()
    
timeshift_no_cuts_3600 = [i + SHIFT_TIME for i in time_no_cuts_3600]
nodeshift_no_cuts_3600 = [j + SHIFT_NODES for j in nodes_no_cuts_3600]
    
timeshift_cuts_3600 = [i + SHIFT_TIME for i in time_cuts_3600]
nodeshift_cuts_3600 = [j + SHIFT_NODES for j in nodes_cuts_3600]

instances_3600 = len(df_3600)

time_no_cuts_3600 = []
nodes_no_cuts_3600 = []
cuts_geom_time_3600 = geometric_mean(timeshift_no_cuts_3600) - SHIFT_TIME
cuts_geom_node_3600 = geometric_mean(nodeshift_no_cuts_3600) - SHIFT_NODES
time_no_cuts_3600.append(cuts_geom_time_3600)
nodes_no_cuts_3600.append(cuts_geom_node_3600)

time_cuts_3600 = []
nodes_cuts_3600 = []
cuts_geom_time_3600 = geometric_mean(timeshift_cuts_3600) - SHIFT_TIME
cuts_geom_node_3600 = geometric_mean(nodeshift_cuts_3600) - SHIFT_NODES
time_cuts_3600.append(cuts_geom_time_3600)
nodes_cuts_3600.append(cuts_geom_node_3600)

instances = []
instances.append(instances_0)
instances.append(instances_10)
instances.append(instances_100)
instances.append(instances_1000)
instances.append(instances_3600)

time_no_cuts = time_no_cuts_0 + time_no_cuts_10 + time_no_cuts_100 + time_no_cuts_1000 + time_no_cuts_3600
node_no_cuts = nodes_no_cuts_0 + nodes_no_cuts_10 + nodes_no_cuts_100 + nodes_no_cuts_1000 + nodes_no_cuts_3600

time_cuts = time_cuts_0 + time_cuts_10 + time_cuts_100 + time_cuts_1000 + time_cuts_3600  
node_cuts = nodes_cuts_0 + nodes_cuts_10 + nodes_cuts_100 + nodes_cuts_1000 + nodes_cuts_3600

print(instances)
print(time_cuts)
print(time_no_cuts)

[71, 69, 64, 53, 41]
[5804.78244324949, 6634.747617464731, 9233.210233931706, 17676.67094863734, 20866.713043930344]
[49418.86721675505, 48567.30561026566, 46281.47221941789, 40311.908925119526, 49799.066841775675]


In [118]:
bb_classes = ['Geometric Mean']
num_bb_classes = len(bb_classes)

bucket_min = [0, 10, 100, 1000, 3600]
bucket_max = [18000, 18000, 18000, 18000, 18000]
num_buckets = len(bucket_min)
assert(len(bucket_max) == num_buckets)
bb_buckets = ['[' + str(bucket_min[j]) + ',' + str(bucket_max[j]) + ')' for j in range(num_buckets)]

bb_metrics = ['Gmean']

In [119]:
avg_bb_cols = pd.MultiIndex.from_product([['Time','Node'],
                                     ['Cut','No Cut']],
                                    names=['Parameter','Setting'])

bb_row_names = pd.MultiIndex.from_product(
    [bb_classes, bb_buckets],
    names=['Metric', 'Bucket'])

avg_bb_df = pd.DataFrame(
    index = bb_row_names,
    dtype = float)

avg_bb_df = pd.DataFrame(
    columns = avg_bb_cols,
    index = bb_row_names,
    dtype = float)

avg_bb_df['Instances'] = instances

avg_bb_df[('Time', 'Cut')] = time_cuts
avg_bb_df[('Time', 'No Cut')] = time_no_cuts
avg_bb_df[('Node', 'Cut')] = node_cuts
avg_bb_df[('Node', 'No Cut')] = node_no_cuts


avg_bb_df

Parameter                            Time                        Node  \
Setting                               Cut        No Cut           Cut   
Metric         Bucket                                                   
Geometric Mean [0,18000)      5804.782443  49418.867217  22935.534813   
               [10,18000)     6634.747617  48567.305610  25242.355748   
               [100,18000)    9233.210234  46281.472219  30767.817567   
               [1000,18000)  17676.670949  40311.908925  49400.431711   
               [3600,18000)  20866.713044  49799.066842  64670.314054   

Parameter                                  Instances  
Setting                             No Cut            
Metric         Bucket                                 
Geometric Mean [0,18000)     216005.162167        71  
               [10,18000)    198538.550113        69  
               [100,18000)   161882.689878        64  
               [1000,18000)  134018.981876        53  
               [3600,18000)  161546.937572        41

In [120]:
# SET 3
# 10 out of 10 runs timeout

set_3_no_cuts = data_no_cuts[(data_no_cuts["SOLUTION TIME"] >= 18000)]
max_repeat = 5
vc = set_3_no_cuts['NAME'].value_counts()
set_3_no_cuts = set_3_no_cuts[set_3_no_cuts['NAME'].isin(vc[vc >= max_repeat].index)]


set_3_cuts = data_cuts[(data_cuts["SOLUTION TIME"] >= 18000)]
max_repeat = 5
c = set_3_cuts['NAME'].value_counts()
set_3_cuts = set_3_cuts[set_3_cuts['NAME'].isin(c[c >= max_repeat].index)]

In [121]:
set_3_no_cuts = set_3_no_cuts.groupby('NAME').mean()
set_3_cuts = set_3_cuts.groupby('NAME').mean()

set_3_no_cuts = set_3_no_cuts[["SOLUTION TIME", "No. OF NODES", 'GAP']]
set_3_cuts = set_3_cuts[["SOLUTION TIME", "No. OF NODES", 'GAP']]

set_3_no_cuts.rename(columns={'SOLUTION TIME': 'TIME W/O CUTS', 'No. OF NODES': 'NODES W/O CUTS', 'GAP': 'GAP W/O CUTS'}, inplace=True)
set_3_cuts.rename(columns={'SOLUTION TIME': 'TIME W/ CUTS', 'No. OF NODES': 'NODES W/ CUTS', 'GAP': 'GAP W/ CUTS'}, inplace=True)

# set_3_no_cuts
# set_3_cuts
analysis_set_3 = set_3_no_cuts.merge(set_3_cuts, on = "NAME", how = "left")
analysis_set_3 = analysis_set_3.merge(collection, on = "NAME", how = "left")
analysis_set_3 = analysis_set_3.dropna()
analysis_set_3

,NAME,TIME W/O CUTS,NODES W/O CUTS,GAP W/O CUTS,TIME W/ CUTS,NODES W/ CUTS,GAP W/ CUTS,Status,Variables
0,2club200v15p5scn.mps.gz,18000.004712,1448589.8,4.282090e-01,18000.000812,303070.0,6.216752e-01,hard,200.0
1,academictimetablebig.mps.gz,18000.047770,124.0,1.000000e+20,18000.087116,6.4,1.000000e+20,hard,168974.0
2,academictimetablesmall.mps.gz,18000.002014,3472.6,1.000000e+20,18000.002546,1528.4,1.000000e+20,easy,28926.0
3,adult-max5features.mps.gz,18000.156680,751.4,8.191312e+04,18000.350648,132.2,1.016838e+03,hard,32674.0
4,allcolor10.mps.gz,18000.003150,123034.2,4.050000e+01,18000.001658,124463.2,4.075500e+01,hard,14872.0
...,...,...,...,...,...,...,...,...,...
265,v150d30-2hopcds.mps.gz,18000.021720,1100738.4,1.876101e-01,18000.018458,1031644.8,1.876782e-01,hard,150.0
266,vpphard.mps.gz,18000.005188,10453.8,8.678266e+00,18000.006550,5591.4,6.742965e+00,easy,51471.0
267,vpphard2.mps.gz,18000.010124,21904.4,7.656876e+00,18000.139756,10738.8,3.103191e+01,easy,199999.0
268,wnq-n100-mw99-14.mps.gz,18000.019266,15860.6,6.390817e-01,18000.009874,1583.2,2.930406e-01,hard,10000.0
